## Lab01: Collect and preprocess data

- Name: Phùng Anh Khoa
- Student code: 19120548

***
## How to do and submit your assignment

**Work on your assignment**

You will do your assignment directly in this notebook. First, fill in your name and ID at the beginning of the file. In the notebook, fill in places that say:
```python
#TODO
```

During your work, you can print out the result, create more cells to test, or create more functions to handle things. Please note that <font color=red>you are not allowed to delete or modify my code cells</font> (except in the case that mentioned above). Let remove `raise NotImplementedError("not implement")` when running code.

Always press `Ctrl + S` in order to save your work.

**Notes:** 

    *  Copy means zero
    *  You have to submit your work on time. No exception
    *  Any questions about your grade are based on the assignment submitted on Moodle
    *  Wrong submission takes you -2 points

**Submission guideline**

When grading your assignment, I will choose `Kernel` - `Restart Kernel & Run All Cells` in order to restart the kernel and run all cells in your notebook. Therefore, you should do that before submitting to ensure that the outputs are all as expected.

After that, you make a submited direction as follow:

- Folder `StudentCode` (e.g. If your student code is 1234567, then your folder is `1234567`)
    - File `<StudentCode>.ipynb` (e.g. If your student code is 1234567, then your file is `1234567.ipynb`)

Finally, you compress your folder (`StudentCode`) and submit on Moodle. **The extension of the file is nothing else but `.zip`.**

<font color=red>Please strictly follow the submission rules.</font>

# 1. Set-up environment

In [1]:
#Necessary Packages
import time
import requests
import numpy as np
import pandas as pd 
from bs4 import BeautifulSoup
import json
# YOUR CODE HERE (OPTION) 
# If you need other support packages

# 2. Collect data from a website by parsing HTML (3p)

In this section, you are going to collect data from a website simulating the sale of Pokemon. I have prepared all the needed links in a file (`pokemon.txt`), and you have to crawl data from these links. The expected output is a `dataframe` with the following fields:

+ `SKU`: ID of Pokemon
+ `Name`: Name of Pokemon
+ `Price`: Price of Pokemon 
+ `InStock`: Quantity of Pokemons in stock
+ `Categories`: The category of Pokemon
+ `Tags`: Tags of Pokemon

Your mission is to complete function `collect_data` with `course_urls_file` as your input parameter. The output should look like `pokemon_example.csv` (I list out some examples so you can easily imagine your work).

In [2]:
pokemon_example = pd.read_csv('pokemon_example.csv')
pokemon_example

,SKU,Name,Price,InStock,Categories,Tags
0,4391,Bulbasaur,63.0,45,"Pokemon, Seed","bulbasaur, Overgrow, Seed"
1,7227,Ivysaur,87.0,142,"Pokemon, Seed","ivysaur, Overgrow, Seed"
2,7036,Venusaur,105.0,30,"Pokemon, Seed","Overgrow, Seed, venusaur"
3,9086,Charmander,48.0,206,"Lizard, Pokemon","Blaze, charmander, Lizard"
4,6565,Charmeleon,165.0,284,"Flame, Pokemon","Blaze, charmeleon, Flame"


In [3]:
def extract_price(price_element):
    # Extract the price from the price_element, handling any extra characters or whitespaces
    price_text = price_element.find('span', class_='woocommerce-Price-amount').text.strip()
    price_text = price_text.replace('£', '')  # Remove currency symbol
    try:
        price = float(price_text)
        price = int(price)
    except ValueError:
        price = 0  # Default to 0 if the price cannot be extracted as a float
    return price

In [4]:
def collect_data(course_urls_file):
    #load paths from file
    url_file = open(course_urls_file)
    urls = url_file.readlines()
    urls_filtered = [item[:-1] for item in urls]
    
    #init empty list to store the values of each attribute.
    SKU = []
    Names = []
    Prices = []
    InStocks = []
    Categories = []
    Tags = []

    for url in urls_filtered:
        #TODO
        # Send an HTTP GET request to the URL
        response = requests.get(url)
        
        if response.status_code == 200:
            # Parse the HTML content of the page
            soup = BeautifulSoup(response.text, 'html.parser')

            # Extract data from the page
            sku = soup.find('span', class_='sku').text
            name = soup.find('h1', class_='product_title').text

            # Extract the price using the extract_price function
            price_element = soup.find('p', class_='price')
            price = extract_price(price_element)
            
    

            # Extract the in-stock quantity
            in_stock_element = soup.find('p', class_='stock in-stock')
            in_stock_text = in_stock_element.text
            in_stock_quantity = int(in_stock_text.split()[0])  # Extract the first part as an integer

            # Extract categories by finding the 'a' tags within the 'posted_in' span
            categories_element = soup.find('span', class_='posted_in')
            categories = ', '.join([a.text for a in categories_element.find_all('a', rel='tag')])

            # Extract tags by finding the 'a' tags within the 'tagged_as' span
            tags_element = soup.find('span', class_='tagged_as')
            tags = ', '.join([a.text for a in tags_element.find_all('a', rel='tag')])

            # Append the extracted data to the respective lists
            SKU.append(sku)
            Names.append(name)
            Prices.append(price)
            InStocks.append(in_stock_quantity)
            Categories.append(categories)
            Tags.append(tags)

        else:
            print(f"Failed to retrieve data from {url}")
    
    data = pd.DataFrame({"SKU": SKU,
                         "Name": Names,
                         "Price": Prices, 
                         "InStock": InStocks,
                         "Categories": Categories,
                         "Tags": Tags})
    
    return data

In [5]:
#TEST
data_pokemon = collect_data("pokemon.txt") 
assert data_pokemon.shape == (755, 6)

In [6]:
#Save to csv file with name pokemon.csv
#TODO
data_pokemon.to_csv('pokemon.csv', index=False)


In [7]:
pokemon = pd.read_csv('pokemon.csv')
pokemon

,SKU,Name,Price,InStock,Categories,Tags
0,4391,Bulbasaur,63,45,"Pokemon, Seed","bulbasaur, Overgrow, Seed"
1,7227,Ivysaur,87,142,"Pokemon, Seed","ivysaur, Overgrow, Seed"
2,7036,Venusaur,105,30,"Pokemon, Seed","Overgrow, Seed, venusaur"
3,9086,Charmander,48,206,"Lizard, Pokemon","Blaze, charmander, Lizard"
4,6565,Charmeleon,165,284,"Flame, Pokemon","Blaze, charmeleon, Flame"
...,...,...,...,...,...,...
750,1160,Marshadow,107,250,"Gloomdweller, Pokemon","Gloomdweller, marshadow, Technician"
751,1771,Poipole,130,21,"Poison Pin, Pokemon","Beast Boost, poipole, Poison Pin"
752,9798,Naganadel,30,255,"Poison Pin, Pokemon","Beast Boost, naganadel, Poison Pin"
753,1947,Stakataka,190,210,"Pokemon, Rampart","Beast Boost, Rampart, stakataka"


# 3. Collect data using Web API (4p)

In this section, your work is to practice to crawl data using Web API (http://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL). This is the data of World Bank which includes demographic data and other statistics related to Population, Employment, Health, GDP, Energy Consumption,... for all countries in the world from 1960 to 2022.

From the following selected indicators:
- `SP.POP.TOTL` - Total population
- `SP.POP.TOTL.FE.IN` - Total female population
- `SP.POP.TOTL.MA.IN` - Total male population
- `SP.DYN.CBRT.IN` - Birth rate
- `SP.DYN.CDRT.IN` - Death rate
- `SP.DYN.LE00.MA.IN` - Average life expectancy of male
- `SP.DYN.LE00.FE.IN` - Average life expectancy of female
- `SE.PRM.ENRR` - Primary school enrollment rate
- `SE.TER.ENRR` - High school enrollment rate
- `SE.PRM.CMPT.ZS` - Primary completion rate
- `SE.ADT.1524.LT.ZS` - Literacy rate of people ages 15-24

You are required to collect data from 7 countries and save to dataframe `data_countries`:
- `US` - United States of America
- `IN` - India
- `CN` - China
- `JP` - Japan
- `CA` - Canada
- `GB` - Great Britain
- `ZA` - South Africa

You can expand your work on collecting data (such as collecting data from other countries and other indicators) by reading: https://datahelpdesk.worldbank.org/knowledgebase/articles/889392-api-documentation

**Hints**:

- Use the based URL: http://api.worldbank.org/v2/
- In order to collect data for each indicator of each country, you can use the URL: "http://api.worldbank.org/v2/countries/{country_code}/indicators/{indicator_code}"
    + `country_code` and `indicator_code` are provided above.
    + For example, you can use the following URL to get the `Total population` of Japan: http://api.worldbank.org/v2/countries/jp/indicators/SP.POP.TOTL

In [8]:
data_countries_examples = pd.read_csv("countries_example.csv")
data_countries_examples

,Total Population,Female Population,Male Population,Birth Rate,Death Rate,Male life expectancy,Female life expectancy,"School enrollment, primary","School enrollment, tertiary",Primary completion rate,Literacy rate,Year,Country
0,333287557.0,168266219.0,165021339.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022,USA
1,332031554.0,167550001.0,164481553.0,11.0,10.400,73.5,79.3,NaN,NaN,NaN,NaN,2021,USA
2,331511512.0,167203010.0,164308503.0,10.9,10.300,74.2,79.9,100.305794,87.567657,100.923668,NaN,2020,USA
3,328329953.0,165599805.0,162730147.0,11.4,8.700,76.3,81.4,100.981300,87.888710,100.489052,NaN,2019,USA
4,326838199.0,164926348.0,161911851.0,11.6,8.678,76.2,81.2,101.256561,88.299179,100.092697,NaN,2018,USA
5,325122128.0,164151818.0,160970309.0,11.8,8.638,76.1,81.1,101.821442,88.167389,98.832199,NaN,2017,USA
6,323071755.0,163224028.0,159847727.0,12.2,8.493,76.1,81.1,101.362862,88.835052,NaN,NaN,2016,USA
7,320738994.0,162158414.0,158580581.0,12.4,8.440,76.3,81.2,100.299911,88.889412,NaN,NaN,2015,USA
8,318386329.0,161084758.0,157301571.0,12.5,8.237,76.5,81.3,99.673378,88.626869,NaN,NaN,2014,USA
9,316059947.0,160034189.0,156025758.0,12.4,8.215,76.4,81.2,99.455437,88.726418,NaN,NaN,2013,USA


In [9]:


################################
BASE_URL = 'http://api.worldbank.org/v2/'
COUNTRIES = ["US", "IN", "CN", "JP", "CA", "GB", "ZA"]
INDICATORS = ['SP.POP.TOTL',
             'SP.POP.TOTL.FE.IN',
             'SP.POP.TOTL.MA.IN',
             'SP.DYN.CBRT.IN',
             'SP.DYN.CDRT.IN',
             'SP.DYN.LE00.MA.IN',
             'SP.DYN.LE00.FE.IN',
             'SE.PRM.ENRR',
             'SE.TER.ENRR',
             'SE.PRM.CMPT.ZS',
             'SE.ADT.1524.LT.ZS']
FORMAT="JSON"

In [10]:
# def collect_data(countryCode, per_page, start_year, end_year):
#     #TODO
#     data = []

#     for indicator in INDICATORS:
#         url = f"{BASE_URL}country/{countryCode}/indicator/{indicator}"
#         params = {
#             'format': 'json',
#             'per_page': per_page,
#             'date': f"{start_year}:{end_year}",
#         }

#         response = requests.get(url, params=params)

#         if response.status_code == 200:
#             data += response.json()[1]

#     # Convert the list of data to a DataFrame
#     data_df = pd.DataFrame(data)

#     return data_df

##################################
def collect_data(countryCode, per_page, start_year, end_year):
    #TODO
    TOTAL_POPULATION=[]
    FEMALE_POPULATION=[]
    MALE_POPULATION=[]
    BIRTH_RATE=[]
    DEATH_RATE=[]
    MALE_LIFE_EXPECTACY=[]
    FEMALE_LIFE_EXPECTACY=[]
    SE_PRIMARY=[]
    SE_TERTIARY=[]
    PRIMARY_COMPLETION_RATE=[]
    LITERACY_RATE=[]
    YEAR=[]
    COUNTRY=[]

    for indicator in INDICATORS:
      page=requests.get(BASE_URL+"countries/"+countryCode+
                        "/indicators/"+indicator+
                        "?per_page="+str(per_page)+
                        "&date="+str(start_year)+":"+str(end_year)+
                        "&format="+FORMAT)
      years=json.loads(page.text)
      countryiso3code=years[1][0]['countryiso3code']
      for year in years[1]:
        #print(year)
        if year['indicator']['id']=='SP.POP.TOTL':
          TOTAL_POPULATION.append(year['value'])
        if year['indicator']['id']=='SP.POP.TOTL.FE.IN':
          FEMALE_POPULATION.append(year['value'])
        if year['indicator']['id']=='SP.POP.TOTL.MA.IN':
          MALE_POPULATION.append(year['value'])
        if year['indicator']['id']=='SP.DYN.CBRT.IN':
          BIRTH_RATE.append(year['value'])
        if year['indicator']['id']=='SP.DYN.CDRT.IN':
          DEATH_RATE.append(year['value'])
        if year['indicator']['id']=='SP.DYN.LE00.MA.IN':
          MALE_LIFE_EXPECTACY.append(year['value'])
        if year['indicator']['id']=='SP.DYN.LE00.FE.IN':
          FEMALE_LIFE_EXPECTACY.append(year['value'])
        if year['indicator']['id']=='SE.PRM.ENRR':
          SE_PRIMARY.append(year['value'])
        if year['indicator']['id']=='SE.TER.ENRR':
          SE_TERTIARY.append(year['value'])
        if year['indicator']['id']=='SE.PRM.CMPT.ZS':
          PRIMARY_COMPLETION_RATE.append(year['value'])
        if year['indicator']['id']=='SE.ADT.1524.LT.ZS':
          LITERACY_RATE.append(year['value'])
    count=0
    for i in range(len(TOTAL_POPULATION)):
      YEAR.append(end_year-count)
      count+=1
    data = pd.DataFrame({"Total Population": TOTAL_POPULATION,
                         "Female Population": FEMALE_POPULATION,
                         "Male Population": MALE_POPULATION,
                         "Birth Rate": BIRTH_RATE,
                         "Death Rate": DEATH_RATE,
                         "Male life expectancy": MALE_LIFE_EXPECTACY,
                         "Female life expectancy":FEMALE_LIFE_EXPECTACY,
                         "School enrollment, primary":SE_PRIMARY,
                         "School enrollment, tertiary":SE_TERTIARY,
                         "Primary completion rate":PRIMARY_COMPLETION_RATE,
                         "Literacy rate":LITERACY_RATE,
                         "Year": YEAR,
                         "Country":countryiso3code})
    data=data.replace(np.nan, None)
    return data



In [11]:
# response = requests.get("https://api.worldbank.org/v2/countries/jp/indicators/SP.POP.TOTL.FE.IN")
# root = ET.fromstring(response.content)

# namespace = {'wb': 'http://www.worldbank.org'}

# #data_elements = root.findall(".//wb:data", namespaces=namespace)
# data_elements = root.findall(".//wb:data",namespaces = namespace)
# for data_element in data_elements:
#         country_element = data_element.find("wb:countryiso3code", namespaces=namespace)
#         date_element = data_element.find("wb:date", namespaces=namespace)
#         birth_rate_element = data_element.find("wb:value", namespaces=namespace)
#         date = date_element.text
#         birth_rate = birth_rate_element.text
#         country = country_element.text

#         print(date,birth_rate,country)

In [12]:
def Generate_Countries_Dataset(countryCode_list):
    data = pd.DataFrame()
    for countryCode in countryCode_list:
        data = pd.concat([data, collect_data(countryCode = countryCode, per_page = 100, start_year = 2000, end_year = 2022)], axis=0)
    return data

In [13]:
#TEST
data_countries = Generate_Countries_Dataset(COUNTRIES)
assert data_countries.shape == (161, 13)

In [14]:
print(data_countries.shape)


(161, 13)


In [15]:
# Save to csv file with name countries.csv
#TODO
data_countries.to_csv('countries.csv', index=False)

In [16]:
countries = pd.read_csv('countries.csv')
countries

,Total Population,Female Population,Male Population,Birth Rate,Death Rate,Male life expectancy,Female life expectancy,"School enrollment, primary","School enrollment, tertiary",Primary completion rate,Literacy rate,Year,Country
0,333287557,168266219,165021339,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022,USA
1,332031554,167550001,164481553,11.000,10.400,73.500,79.300,NaN,NaN,NaN,NaN,2021,USA
2,331511512,167203010,164308503,10.900,10.300,74.200,79.900,100.305794,87.567657,100.923668,NaN,2020,USA
3,328329953,165599805,162730147,11.400,8.700,76.300,81.400,100.981300,87.888710,100.489052,NaN,2019,USA
4,326838199,164926348,161911851,11.600,8.678,76.200,81.200,101.256561,88.299179,100.092697,NaN,2018,USA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,48556071,25379489,23176582,21.261,12.947,51.891,55.721,106.446632,NaN,95.654922,NaN,2004,ZAF
157,48104048,25178665,22925383,20.473,12.657,51.970,56.232,105.426369,NaN,90.579948,NaN,2003,ZAF
158,47661514,24979734,22681781,20.040,11.784,53.191,57.668,103.968452,NaN,86.340828,NaN,2002,ZAF
159,47229714,24781908,22447806,20.392,10.871,54.565,59.470,101.990463,NaN,NaN,NaN,2001,ZAF
